<a href="https://colab.research.google.com/github/tarboledah/SyS/blob/main/Parcial2/Parcial_punto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Punto 3

In [9]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz


up to date, audited 23 packages in 415ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cach

In [10]:
!pip install soundfile

In [11]:
#Instalacion del download de youtube
import os
import yt_dlp as youtube_dl
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

In [12]:
link_= ["https://youtu.be/wmef2TEPGgo?si=XveKGLNLU5Brstup"] #Se selecciona el link de youtube de la canciòn deseada
name_ = ["Stupid"]                                        #Nombre de la banda/artista
for name,link in zip(name_,link_):
    print(name)
    download_ytvid_as_mp3(link,name)                        #Se descarga el audio mp3 de la cancion

Stupid
[youtube] Extracting URL: https://youtu.be/wmef2TEPGgo?si=XveKGLNLU5Brstup
[youtube] wmef2TEPGgo: Downloading webpage
[youtube] wmef2TEPGgo: Downloading ios player API JSON
[youtube] wmef2TEPGgo: Downloading web creator player API JSON
[youtube] wmef2TEPGgo: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=wmef2TEPGgo
[youtube] wmef2TEPGgo: Downloading webpage
[youtube] wmef2TEPGgo: Downloading ios player API JSON
[youtube] wmef2TEPGgo: Downloading web creator player API JSON
[youtube] wmef2TEPGgo: Downloading m3u8 information
[info] wmef2TEPGgo: Downloading 1 format(s): 251
[download] Destination: Stupid.mp3
[download] 100% of    4.64MiB in 00:00:00 at 11.51MiB/s  
Download complete... Stupid.mp3


In [13]:
!ffmpeg -y -i Stupid.mp3 output.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [14]:
%%writefile Parcial_punto3.py
#Codigo del Dashboard
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import yt_dlp as youtube_dl
import joblib
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

st.set_page_config(
    page_title="Punto3",
    page_icon="3️⃣",
)

st.title("Punto 3")
st.header("Frecuencia de la señal modelada en amplitud")

nombre_out = "output.wav"
audio = "/content/output.wav"
song_duration = st.slider('Segundos de duración:')
st.audio(audio, format='audio/wav', start_time=0, end_time=song_duration)
x, fs = sf.read(nombre_out)
# read speech signal from file
print('Frecuencia de muestreo %.2f[Hz]\naudio %s' % (fs,nombre_out))

# Cargar el archivo de audio
audio_signal, sampling_rate = sf.read(nombre_out)

# Verificar cuántos canales tiene el archivo
print(f"Forma del audio: {audio_signal.shape}")

# Si es estéreo (2 canales), lo convertimos en mono promediando los canales
if len(audio_signal.shape) > 1 and audio_signal.shape[1] == 2:
    audio_mono = np.mean(audio_signal, axis=1)
else:
    audio_mono = audio_signal  # Si ya es mono, no hacemos nada

# Guardar el archivo de audio en mono
sf.write('audio_mono.wav', audio_mono, sampling_rate)

audio_mono.shape

ti, tf = st.slider('Seleccione el rango de procesamiento',max_value=120,value=(20, 25))

xpro = audio_mono.copy() #copiar archivos para procesar
#ti = 20 #tiempo incio a procesar seg
#tf = 25 #tiempo final a procesar seg
xs = xpro[int(ti*fs):int((tf*fs))]

from IPython.display import Audio
ns = 120 #se reproducen los primeros ns segundos
Audio(xs[:int(fs*ns)].T,rate=fs)

audio_mono = audio_mono / np.max(np.abs(audio_mono))
xs= xs/np.max(np.abs(xs))

# Simulación de modulación por amplitud (AM)
def am_modulate(audio_signal, carrier_freq, sampling_rate, mod_index):
    t = np.arange(len(audio_signal)) / sampling_rate
    carrier = np.sin(2 * np.pi * carrier_freq * t)
    am_signal = (1 + mod_index * audio_signal) * carrier
    return t, carrier, am_signal

#Simulación de la modulación AM
carrier_freq = 100  #Frecuencia de la portadora
mod_index = float(st.number_input("Ingrese el indice de modulacion:",value=10.1))   # Índice de modulación
t, carrier, am_signal = am_modulate(audio_mono, carrier_freq, sampling_rate, mod_index)

def plot_signal(time, signal, title):
    plt.figure(num=1, figsize=(10, 4))
    plt.plot(time, signal)
    plt.title(title)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    plt.grid()
    st.pyplot()

def plot_frequency(signal, sampling_rate, title):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)[:N // 2]
    plt.figure(num=2, figsize=(10, 4))
    plt.plot(xf, 2.0 / N * np.abs(yf[:N // 2]))
    plt.title(title)
    plt.grid()
    st.pyplot()

st.subheader("Graficas de la señal")

plot_signal(t, audio_mono, "Señal de Mensaje (Tiempo)")
plot_signal(t, carrier, "Señal Portadora (Tiempo)")
plot_signal(t, am_signal, "Señal Modulada AM (Tiempo)")

#Transformada de Fourier para mostrar en frecuencia
plot_frequency(audio_mono, sampling_rate, 'Señal de Mensaje (Frecuencia)')
plot_frequency(carrier, sampling_rate, 'Señal Portadora (Frecuencia)')
plot_frequency(am_signal, sampling_rate, 'Señal Modulada AM (Frecuencia)')

 #Reproducir los audios
st.header("Señales resultantes")
play, fin = st.slider('Seleccione el rango de Reproduccion',max_value=120,value=(20, 25))

#Señal En Mono
st.audio(xs, sample_rate=fs)

#Señal Portadora
xpro_1 = carrier.copy() #copiar archivos para procesar
#ti = 20 #tiempo incio a procesar seg
#tf = 25 #tiempo final a procesar seg
xs_1 = xpro_1[int(play*fs):int((fin*fs))]
st.audio(xs_1, sample_rate=fs)

#Señal Modulada
xpro_2= am_signal.copy() #copiar archivos para procesar
#ti = 20 #tiempo incio a procesar seg
#tf = 25 #tiempo final a procesar seg
xs_2 = xpro_2[int(play*fs):int((fin*fs))]
st.audio(xs_2, sample_rate=fs)


Overwriting Parcial_punto3.py


In [15]:
!streamlit run Parcial_punto3.py &>/content/logs.txt & curl ipv4.icanhazip.com #Password del Dashboard

34.16.130.167


In [ ]:
!npx localtunnel --port 8501 #Link del Dashboard

your url is: https://tame-zebras-punch.loca.lt
